![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# Размышления над выполненой работой

>> Что получилось

* Результат 0,21173 на лидерборде
* Получил опыт предобработки данных
* Познакомился с работой на kaggle, интересная платформа
* Изучил несколько работ других участников соревнования, познавательно
* При заполнении пропусков константой, наиболее лучший результат получился при заполнении средним
* В одном случае заполнение нулем было лучше чем средним, пока непонятно почему оставил на потом подумать
* Осознал что простое заполнение пропусков константой не самый эффективный вариант, обращу внимание на это
* Удалось создать парочку признаков которые улучшили модель
* На несколько дней погрузился в работу и получил удовлетворение от того что удалось немного улучшить модель
* Профессия датасаентиста очень сложная но интересная!

>> Что не получилось
* Как и в предыдущий раз мало дней оставил на финальную работу
* Совершенно не учитывал результаты EDA
* Некоторые признаки ухудшали модель, почему оставил на подумать

Весь код поместил в предложенную функцию предобработки... просто потому что до строки где рассчитывается МАЕ меньше проматывать код.

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
from datetime import datetime, timedelta
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

In [ ]:
data['Number of Reviews'].mean()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
data['Number_of_Reviews_isNAN']

In [ ]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [ ]:
data.head(5)

In [ ]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [ ]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
data.info()

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [ ]:
# тут ваш код на обработку других признаков
# .....

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    # пробовал заполнить средним значением, МАЕ становится хуже чем с нулем.
    df_output['Number of Reviews'].fillna(0, inplace=True)
        
    # тут ваш код по обработке NAN
    
    # пропуски заполняем 0, с ним результат чуть лучше
    # сперва пробовал заполнить "$$ - $$$", МАЕ получался хуже
    df_output['Price Range'] = df_output['Price Range'].apply(lambda x: str(x).replace('nan', '0'))
    
    # предположил что там где пропуски какая то своя кухня, назвал ее "other"
    df_output['Cuisine Style'] = df_output['Cuisine Style'].fillna('other')
    
    # в тестовой выборке присутствует 2 пропуска, заменим их на пустую конструкцию чтобы не вылетала ошибка
    df_output['Reviews'].fillna('[[], []]', inplace=True)
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=['City',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    # ....
    # обрабатывем столбец Price Range
    df_output['Price Range'] = df_output['Price Range'].apply(lambda x: str(x).replace('$$ - $$$', '2'))
    df_output['Price Range'] = df_output['Price Range'].apply(lambda x: str(x).replace('$$$$', '3'))
    df_output['Price Range'] = df_output['Price Range'].apply(lambda x: str(x).replace('$', '1'))
    df_output['Price Range'] = df_output['Price Range'].astype(int)
    
    # Если взять логарифм от столбца то МАЕ улучшается
    df_output.Ranking = df_output.Ranking.apply(lambda x: math.log(x))
    df_output['Number of Reviews'] = df_output['Number of Reviews'].apply(lambda x: math.log(x+1))
    # Делаем копию столбца 'Cuisine Style' и сплитим для создания нового признака количества кухонь
    df_output['Cuisine_Counts'] = df_output['Cuisine Style'].apply(lambda x: str(x).split(','))
    
    # попробую столбец ID_TA сделать числовым, МАЕ ухудшился, закоментарю код
    #df_output.ID_TA = df_output.ID_TA.apply(lambda x: re.sub("[^0-9]","",x))
    #df_output.ID_TA = df_output.ID_TA.astype(int)
        
    # обрабатывем столбец Cuisine Style
    # сперва делаем очистку данных от лишних символов
    # затем копируем столбец в отдельную переменную, к ней применяем str.get_dummies
    # получившийся набор обьединяем с нашим исходным
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda x: str(x).replace(' ', ''))
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda x: str(x).replace('[', ''))
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda x: str(x).replace(']', ''))
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda x: re.sub("[^A-Z,a-z|^,]","",x))
    data_Cuisine = df_output['Cuisine Style'].copy()
    data_Cuisine = data_Cuisine.str.get_dummies(sep=',')
    df_output = pd.concat([df_output, data_Cuisine], axis=1)
        
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # из столбца 'Reviews' выделим даты отзывов и найдем кол-во дней с самого последнего 
    str_data = '01/01/2021'
    dt = datetime.strptime(str_data, '%m/%d/%Y')
    df_output.Reviews = df_output.Reviews.apply(lambda x: str(x).replace('[', ''))
    df_output.Reviews = df_output.Reviews.apply(lambda x: str(x).replace(']', ''))
    df_output.Reviews = df_output.Reviews.apply(lambda x: str(x).split(','))
    df = pd.DataFrame({'Reviews_1':[], 'Reviews_2':[]})
    df.Reviews_1 = df_output.Reviews.apply(lambda x: x[-2])
    df.Reviews_2 = df_output.Reviews.apply(lambda x: x[-1])
    df.Reviews_1 = df.Reviews_1.apply(lambda x: re.sub("[^0-9|^/]","",x))
    df.Reviews_2 = df.Reviews_2.apply(lambda x: re.sub("[^0-9|^/]","",x))
    df.Reviews_1 = df.Reviews_1.apply(lambda x: None if x.strip() == '' else x)
    df.Reviews_2 = df.Reviews_2.apply(lambda x: None if x.strip() == '' else x)
    df_output['Difference_day'] = pd.to_datetime(df.Reviews_1, errors='coerce', format='%m/%d/%Y') - pd.to_datetime(df.Reviews_2, errors='coerce', format='%m/%d/%Y')
    df_output['Difference_day'] = df_output['Difference_day'].dt.days
    df_output['Difference_day'].fillna(124, inplace=True)
    df.Reviews_1 = dt - pd.to_datetime(df.Reviews_1, errors='coerce', format='%m/%d/%Y')
    df.Reviews_2 = dt - pd.to_datetime(df.Reviews_2, errors='coerce', format='%m/%d/%Y')
    df.Reviews_1 = df.Reviews_1.dt.days
    df.Reviews_2 = df.Reviews_2.dt.days
    # выбираем более меньшее количество дней т.е. более свежий отзыв
    df_output['Reviews_day'] = df.apply(lambda row: row['Reviews_1'] if row['Reviews_1']<row['Reviews_2'] else row['Reviews_2'], axis=1)
    # заполняем пропуски в новом признаке чуть меньше среднего значения
    df_output['Reviews_day'].fillna(1300, inplace=True)
    
    # генерим новый признак количество кухонь в ресторане. Очень незначительно улучшил МАЕ
    df_output['Cuisine_Counts'] = df_output['Cuisine_Counts'].apply(lambda x: len(x))
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
